In [1]:
import glob
import json
import pandas as pd
import numpy as np
import os
from pprint import pprint as pp

In [2]:
os.system("rm -rf ./template")
os.system("unzip template.zip")
os.system("tree")

0

In [3]:
files = sorted([intent for intent in glob.glob('template/intents/*.json')])
files.remove('template/intents/Default Fallback Intent.json')
# files.remove('template/intents/consideration-no_usersays_en.json')

In [4]:
# does not assume that all intents have usersays
# assume English only
intent_hold = "none" 
intent_jsons = []
for file in files:
    intent_name = file.rpartition('.')[0]
    if not intent_name.startswith(intent_hold):
        intent_jsons.append([file])
        intent_hold = intent_name
    else:
        intent_jsons[-1].append(file)

In [5]:
intent_jsons

[['template/intents/Default Welcome Intent.json',
  'template/intents/Default Welcome Intent_usersays_en.json'],
 ['template/intents/make chatbot.json',
  'template/intents/make chatbot_usersays_en.json'],
 ['template/intents/make coffee.json',
  'template/intents/make coffee_usersays_en.json'],
 ['template/intents/make love.json',
  'template/intents/make love_usersays_en.json'],
 ['template/intents/make money.json',
  'template/intents/make money_usersays_en.json'],
 ['template/intents/make tea.json',
  'template/intents/make tea_usersays_en.json'],
 ['template/intents/make trouble.json',
  'template/intents/make trouble_usersays_en.json']]

In [6]:
intents = []
for intent_json in intent_jsons:
    intent = {}
    intent_info_json = intent_json[0]
    usersays_jsons = intent_json[1:] # should be one only
    
    with open(intent_info_json) as f:
        intent_info = json.load(f)
    intent["INTENT_NAME"] = intent_info["name"]
    intent["INPUT_CONTEXT"] = intent_info["contexts"]
    intent["OUTPUT_CONTEXT"] = [str(c["lifespan"]) + ", " + str(c["name"])  
                                for c in intent_info["responses"][0]["affectedContexts"]]
    
    response_list_or_str = intent_info["responses"][0]["messages"][0]["speech"]
    if type(response_list_or_str) == str:
        intent["RESPONSES"] = [response_list_or_str]
    else: 
        intent["RESPONSES"] = [m for m in response_list_or_str]

    intent["USER_SAYS"] = []
    for usersays_json in usersays_jsons: # should be one or none
        with open(usersays_json) as f:
            usersays_info = json.load(f)
        for usersay in usersays_info:
            intent["USER_SAYS"].append(usersay["data"][0]["text"])
    intents.append(intent)
#     pp(intent_info)
#     break

In [7]:
intents

[{'INTENT_NAME': 'Default Welcome Intent',
  'INPUT_CONTEXT': [],
  'OUTPUT_CONTEXT': ['3, init'],
  'RESPONSES': ['Welcome to ABC law partners. \nWe can help explore whether you have a legal case against someone who has broken an agreement. \nI am not a lawyer, but I provide preliminary analysis based on the facts of your case. This service is free.\nCan you briefly explain your issue? (e.g. my customer did not pay me, the goods was defective)'],
  'USER_SAYS': ['just going to say hi',
   'heya',
   'hello hi',
   'howdy',
   'hey there',
   'hi there',
   'greetings',
   'hey',
   'long time no see',
   'hello',
   "lovely day isn't it",
   'I greet you',
   'hello again',
   'hi',
   'hello there',
   'a good day']},
 {'INTENT_NAME': 'make chatbot',
  'INPUT_CONTEXT': [],
  'OUTPUT_CONTEXT': [],
  'RESPONSES': ['dashboard.botbot.ai', 'console.botbot.ai'],
  'USER_SAYS': ['i want chatbot']},
 {'INTENT_NAME': 'make coffee',
  'INPUT_CONTEXT': ['thirsty'],
  'OUTPUT_CONTEXT': ['0, thir

In [8]:
len_INPUT_CONTEXT = max([len(intent["INPUT_CONTEXT"]) for intent in intents]+[1])
len_OUTPUT_CONTEXT = max([len(intent["OUTPUT_CONTEXT"]) for intent in intents]+[1])
len_USER_SAYS = max([len(intent["USER_SAYS"]) for intent in intents]+[1])
len_RESPONSES = max([len(intent["RESPONSES"]) for intent in intents]+[1])
lengths = [1, len_INPUT_CONTEXT, len_INPUT_CONTEXT, len_USER_SAYS, len_RESPONSES]

In [9]:
row_length = sum(lengths)
columns = [""]*row_length
columns[0] = "INTENT_NAME"
columns[sum(lengths[:1])] = "INPUT_CONTEXT"
columns[sum(lengths[:2])] = "OUTPUT_CONTEXT"
columns[sum(lengths[:3])] = "USER_SAYS"
columns[sum(lengths[:4])] = "RESPONSES"

In [10]:
row_list = []
for intent in intents:
    row = [np.nan]*row_length
    row[0] = intent["INTENT_NAME"]
    row[sum(lengths[:1]):sum(lengths[:1])+len(intent["INPUT_CONTEXT"])] = intent["INPUT_CONTEXT"]
    row[sum(lengths[:2]):sum(lengths[:2])+len(intent["OUTPUT_CONTEXT"])] = intent["OUTPUT_CONTEXT"]
    row[sum(lengths[:3]):sum(lengths[:3])+len(intent["USER_SAYS"])] = intent["USER_SAYS"]
    row[sum(lengths[:4]):sum(lengths[:4])+len(intent["RESPONSES"])] = intent["RESPONSES"]
    row_list.append(row)

In [11]:
df = pd.DataFrame(row_list, columns=columns)
df

,INTENT_NAME,INPUT_CONTEXT,OUTPUT_CONTEXT,USER_SAYS,,,,,,,...,,,,,,RESPONSES,,,,
0,Default Welcome Intent,NaN,"3, init",just going to say hi,heya,hello hi,howdy,hey there,hi there,greetings,...,I greet you,hello again,hi,hello there,a good day,Welcome to ABC law partners. \nWe can help exp...,NaN,NaN,NaN,NaN
1,make chatbot,NaN,NaN,i want chatbot,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,dashboard.botbot.ai,console.botbot.ai,NaN,NaN,NaN
2,make coffee,thirsty,"0, thirsty",i want coffee,lim kopi ai mai,kopi siu dai gam xia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,half full drink up!,bo ta bo lam pah,bottoms up!,drink till you drop,now converting coffee into code
3,make love,rich,NaN,i want love,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,<3,NaN,NaN,NaN,NaN
4,make money,NaN,"2, thirsty",i want money,bo lui ah,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,huat ah,NaN,NaN,NaN,NaN
5,make tea,thirsty,"10, woke",i want tea,lim tehsi,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,slowly drink,take your time,NaN,NaN,NaN
6,make trouble,NaN,NaN,kua simi kua,le kua si mi,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,come fight la,NaN,NaN,NaN,NaN


In [12]:
df.to_csv("template.csv")

In [13]:
# convert this notebook into a python script
# and then remove the last few lines
get_ipython().system('jupyter nbconvert --to script df-to-csv.ipynb')
os.system("for i in {1..10}; do sed -i '' -e '$ d' df-to-csv.py; done")

[NbConvertApp] Converting notebook df-to-csv.ipynb to script
[NbConvertApp] Writing 3525 bytes to df-to-csv.py


0